###### Data Jobs Analysis 
[Go back to the main page](./Data_Jobs_Analysis.ipynb)

---------

## Data Offers Heatmap
-------


In [1]:
import itertools
import datetime as dt
import psycopg2
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.offline.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline

%matplotlib notebook
%matplotlib inline

In [2]:
conn = psycopg2.connect('dbname=jobsbg')
daily_df = pd.read_sql_query('SELECT * FROM v_data_offers_count_daily', conn, index_col='subm_date')
conn.close()

In [3]:
daily_df.index = pd.to_datetime(daily_df.index)
daily_df.head()


,subm_count
subm_date,
2017-09-27,4
2017-09-28,4
2017-09-29,0
2017-09-30,0
2017-10-01,0


In [4]:
min_ts = min(daily_df.index)
max_ts = max(daily_df.index)
idx = pd.date_range(
    min_ts - dt.timedelta(days=min_ts.weekday()),
    max_ts + dt.timedelta(days=6-max_ts.weekday()))

In [5]:
daily_df = daily_df.reindex(idx)
daily_df.head()

,subm_count
2017-09-25,NaN
2017-09-26,NaN
2017-09-27,4.0
2017-09-28,4.0
2017-09-29,0.0


In [6]:
matrix_df = pd.DataFrame(daily_df.values.reshape(len(daily_df)//7, 7), columns=daily_df.index[:7].strftime('%A'))
matrix_df = matrix_df.T
matrix_df.head(7)

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
Monday,NaN,3.0,4.0,4.0,1.0,7.0,4.0,4.0,3.0,5.0,...,9.0,10.0,8.0,12.0,11.0,17.0,4.0,9.0,7.0,0.0
Tuesday,NaN,10.0,2.0,4.0,8.0,1.0,5.0,4.0,5.0,5.0,...,6.0,7.0,7.0,5.0,1.0,6.0,9.0,5.0,5.0,0.0
Wednesday,4.0,2.0,4.0,5.0,5.0,1.0,3.0,4.0,2.0,5.0,...,7.0,7.0,9.0,3.0,4.0,3.0,5.0,4.0,4.0,NaN
Thursday,4.0,5.0,2.0,3.0,4.0,2.0,3.0,2.0,2.0,1.0,...,8.0,5.0,2.0,2.0,6.0,9.0,2.0,2.0,11.0,NaN
Friday,0.0,4.0,1.0,6.0,4.0,3.0,3.0,4.0,8.0,5.0,...,9.0,3.0,7.0,9.0,6.0,3.0,3.0,7.0,7.0,NaN
Saturday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Sunday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [7]:
matrix_df.columns


RangeIndex(start=0, stop=66, step=1)

In [8]:
vals = [
    [v for v in matrix_df.loc['Monday'].values],
    [v for v in matrix_df.loc['Tuesday'].values],
    [v for v in matrix_df.loc['Wednesday'].values],
    [v for v in matrix_df.loc['Thursday'].values],
    [v for v in matrix_df.loc['Friday'].values],
    [v for v in matrix_df.loc['Saturday'].values],
    [v for v in matrix_df.loc['Sunday'].values]
]

In [9]:
hm = go.Heatmap(
    x = [w for w in daily_df.columns],
    y = [d for d in matrix_df.index],
    z = vals,
    colorscale=[
        [0.0, '#faf0dc'], 
        [0.1111111111111111, '#f2e6ce'], 
        [0.2222222222222222, '#ecd5bb'], 
        [0.3333333333333333, '#e6c4a9'], 
        [0.4444444444444444, '#e1b396'], 
        [0.5555555555555556, '#dba284'], 
        [0.6666666666666666, '#d59171'], 
        [0.7777777777777778, '#d0805f'], 
        [0.8888888888888888, '#ca6f4c'], 
        [1.0, '#c45e3a']],
    xgap = 0,
    ygap = 0,
)  

In [10]:
data = [hm]

In [24]:
layout = go.Layout(
    paper_bgcolor = '#FFFAF0',            
    plot_bgcolor = '#FFFAF0',
    title = 'Data Job Offers Heatmap',
    titlefont = dict(
        size = 22,
        color = '#594B42',
        family = 'FiraSansCondensedRegular',
    ),
    font = dict(
        size = 12,
        color = '#594B42',
        family = 'FiraSansCondensedRegular',
    ),
    xaxis = dict(
        ticks = '',
    ),
    yaxis = dict(
        title = 'Day of week',
        titlefont = dict(
            size = 14,
            color = '#594B42',
            family = 'FiraSansCondensedRegular',
        ),
    ),
)

In [25]:
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig, filename = 'data_jobs_subm_heatmap.html')

In [26]:
# Uncomment the line below to export an HTML version of the chart.
# plotly.offline.plot(fig, filename = 'data_offers_over_time.html')

In [14]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)

In [ ]:
pallete_reds = ["#fffaf0",
"#faf0dc",
"#f2e6ce",
"#ecd5bb",
"#e6c4a9",
"#e1b396",
"#dba284",
"#d59171",
"#d0805f",
"#ca6f4c",
"#c45e3a",
"#bf4d28",
"#af4624",
"#9f4021",
"#8f391e",
"#7f331b"]

pallete_greens = ["#fffaf0",
"#f0f3e7",
"#e2ecde",
"#d4e5d6",
"#c6decd",
"#b8d7c5",
"#aad0bc",
"#9cc9b4",
"#8ec2ab",
"#80bca3",
"#75ac95",
"#6a9c87",
"#5f8c79",
"#557c6c"]